In [ ]:
pip install langchain langgraph


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [462]:
pip install langchain-google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.3.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
from dotenv import load_dotenv
load_dotenv()

from langchain.chat_models import init_chat_model
from langgraph.graph import StateGraph, START, END
from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph.message import add_messages
from langgraph.types import interrupt, Command
from langgraph.checkpoint.memory import MemorySaver

In [464]:
class workflowState(TypedDict):
    preferences: list[str]
    breakfast: list[str]
    lunch: list[str]
    dinner: list[str]

In [465]:
llm = init_chat_model("google_genai:gemini-2.0-flash")

In [466]:
def get_preferences(state: workflowState):
    state["preferences"] = "vegetarian"
    return { "preferences":  state["preferences"] }

In [467]:
def gen_breakfast(state: workflowState):
    # result = llm.invoke(f"generate {state["preferences"]} breakfast meal coma seperated small list")
    state["breakfast"] = "result.content" + "->a"
    return { "breakfast":  state["breakfast"] }

In [468]:
def gen_lunch(state: workflowState):
    # result = llm.invoke(f"generate {state["preferences"]} lunch meal coma seperated small list")
    state["lunch"] = "result.content"
    return { "lunch":  state["lunch"] }

In [469]:
def gen_dinner(state: workflowState):
    # result = llm.invoke(f"generate {state["preferences"]} dinner meal coma seperated small list")
    state["dinner"] = "result.content"
    return Command(
        goto='review',
        update={ "dinner":  state["dinner"] }
    )
    

In [470]:
def review(state: workflowState):
    print("\n Generated meal plan")
    print("Breakfast:", state["breakfast"])
    print("Lunch:", state["lunch"])
    print("Dinner:", state["dinner"])

    data = f"Generated mean plan\n Breakfast: {state["breakfast"]}, Lunch: {state["lunch"]}, Dinner: {state["dinner"]} review and approve the meal"
    decision = interrupt(data)
    if decision == "yes":
        return Command(
            goto="finalize"
        )
    else:
        nodevalue = input('Enter the step you want to go')
        return Command(
            goto=nodevalue
        )

    

In [471]:
def finalize(state: workflowState):
    print(f"meal finalized-->   Breakfast: {state["breakfast"]}, Lunch: {state["lunch"]}, Dinner: {state["dinner"]}")
    return state

In [473]:
graph = StateGraph(workflowState)

memory = MemorySaver()
graph.add_node("get_preferences", get_preferences)
graph.add_node("gen_breakfast", gen_breakfast)
graph.add_node("gen_lunch", gen_lunch)
graph.add_node("gen_dinner", gen_dinner)
graph.add_node("review", review)
graph.add_node("finalize", finalize)
# graph.add_node("reject", reject)

graph.add_edge(START, "get_preferences")
graph.add_edge("get_preferences", "gen_breakfast")
graph.add_edge("get_preferences", "gen_lunch")
graph.add_edge("get_preferences", "gen_dinner")

graph.add_edge("gen_breakfast", "review")
graph.add_edge("gen_lunch", "review")
graph.add_edge("gen_dinner", "review")

graph.add_edge("review", "finalize")
graph.add_edge("finalize", END)

workflow = graph.compile(checkpointer=memory)

In [474]:

config = {"configurable": {"thread_id": "1"}}
result = workflow.invoke({}, config=config, stream_mode="updates")
result


 Generated meal plan
Breakfast: result.content->a
Lunch: result.content
Dinner: result.content


[{'get_preferences': {'preferences': 'vegetarian'}},
 {'gen_breakfast': {'breakfast': 'result.content->a'}},
 {'gen_dinner': {'dinner': 'result.content'}},
 {'gen_lunch': {'lunch': 'result.content'}},
 {'__interrupt__': (Interrupt(value='Generated mean plan\n Breakfast: result.content->a, Lunch: result.content, Dinner: result.content review and approve the meal', resumable=True, ns=['review:790c8322-3ae4-c741-a683-25b3853ca582']),)}]

In [475]:
snapshot = workflow.get_state(config)
snapshot.next

('review',)

In [ ]:
for chunk in workflow.stream({}, config=config, stream_mode="updates"):
    for node_id, value in chunk.items():

        if(node_id == "__interrupt__"):
            while True: 
                decision = input("Approve meal plan? (yes/no): ").strip().lower()

                if decision.lower() == "yes":
                    result = workflow.invoke(Command(resume=decision), config=config, stream_mode="updates")
                    print(result)
                    break
                else:
                    value = input('Eneter the node name you want to go')
                    result = workflow.invoke(Command(goto=value), config=config, stream_mode="updates")
                    print(result)


 Generated meal plan
Breakfast: result.content->a
Lunch: result.content
Dinner: result.content



 Generated meal plan
Breakfast: result.content->a
Lunch: result.content
Dinner: result.content
[{'gen_dinner': {'dinner': 'result.content'}}, {'__interrupt__': (Interrupt(value='Generated mean plan\n Breakfast: result.content->a, Lunch: result.content, Dinner: result.content review and approve the meal', resumable=True, ns=['review:08fd6fda-0e2a-e208-441f-86316a1813d9']),)}]

 Generated meal plan
Breakfast: result.content->a
Lunch: result.content
Dinner: result.content
[{'gen_dinner': {'dinner': 'result.content'}, '__metadata__': {'cached': True}}, {'gen_breakfast': {'breakfast': 'result.content->a'}}, {'__interrupt__': (Interrupt(value='Generated mean plan\n Breakfast: result.content->a, Lunch: result.content, Dinner: result.content review and approve the meal', resumable=True, ns=['review:08fd6fda-0e2a-e208-441f-86316a1813d9']),)}]

 Generated meal plan
Breakfast: result.content->a
Lunch: result.content
Dinner: result.content
meal finalized-->   Breakfast: result.content->a, Lunch: 